In [1]:
# Fastbook setup
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 84.6 MB/s eta 0:00:00
Mounted at /content/gdrive


In [10]:
#hide
from fastbook import *
from fastai.text.all import *
from IPython.display import display,HTML
from pathlib import Path
import pandas as pd

O objetivo deste notebook é criar um modelo capaz de receber como entrada a interação de um cliente com um
vendedor e então dizer qual sua intenção. Permitindo, dessa forma, a criação de mercados digitais com interação por voz.
A fim de criar o modelo de classificação de interações comerciais, é necessário passar por algumas etapas, sendo elas:

1° O treinamento de um modelo de processamento de linguagem natural capaz de prever a próxima palavra de uma determinada frase.

Isto será feito utilizando-se do dataset WikiText-2, que é composto por um conjunto de artigos da wikipedia. Este dataset em especial será utilizado por fornecer ao modelo um amplo vocabulário.

2° O ajuste fino do modelo de NLP treinado na etapa anterior para o desenvolvimento de um modelo mais especializado, capaz de prever a próxima palavra de uma interação comercial da perspectiva de um cliente.

Isto será feito por meio de um dataset próprio que possui frases de diferentes tipos de interação comercial, sendo eles: o pedido de algum produto, a requisição de devolução de um produto, o pedido das opções de produtos disponíveis, e interações de saudação e despedida.

3° O ajuste fino do modelo do passo 2 para criar um modelo capaz de classificar uma determinada interação comercial dentro das categorias já listadas.

### Treinando modelo com o dataset wikitext-2

In [11]:
# Caminho para o conjunto de dados Wikitext-2
path = untar_data(URLs.WIKITEXT_TINY)

# Ler os dados dos arquivos CSV
df_train = pd.read_csv(path/'train.csv')

In [12]:
# Renomear a primeira coluna do dataframe
df_train.rename(columns={df_train.columns[0] : 'train'}, inplace=True)

In [13]:
# Criar Dataloaders para o conjunto de treinamento
dls_train = TextDataLoaders.from_df(df_train, text_col='train', is_lm=True, bs=128, seq_len=80)

In [14]:
# Criar o modelo de linguagem
learn = language_model_learner(dls_train, AWD_LSTM, drop_mult=0.3,
                               metrics=[accuracy, Perplexity()]).to_fp16()

In [15]:
# Treinar o modelo de linguagem
learn.fit_one_cycle(7, 3e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.226153,3.202698,0.407666,24.598808,02:14
1,3.037235,3.104787,0.409239,22.304470,02:08
2,2.874897,3.112789,0.407964,22.483662,02:07
3,2.743517,3.127358,0.406895,22.813631,02:10
4,2.701494,3.135628,0.406182,23.003088,02:08
5,2.662250,3.139539,0.405848,23.093229,02:05
6,2.652601,3.140287,0.405707,23.110491,02:10


In [16]:
# Salvar o modelo pré-treinado
learn.save_encoder('colab_encoder_01')

In [17]:
# Teste de predição
TEXT = "That animal is dangerous because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75)
         for _ in range(N_SENTENCES)]

print('\n'.join(preds))

That animal is dangerous because of the similar nature of its qualities , it is often considered one of the most dangerous animals . The presence of the word " ocean " in the Americas has been cited as a major factor in
That animal is dangerous because of its sexual attraction to humans , it shared between the two species of Chinese gorilla . In June 2011 , American biologist Paul W. Young formulated the African American


### Ajuste fino do modelo pré-treinado para os dados de interação comercial

In [18]:
# Função para obter os arquivos de texto de todas as categorias
get_txt = partial(get_text_files, folders=['BUY', 'REFUND', 'GREETING', "GOODBYE", "LIST"])

In [24]:
# Path dos dados a serem utilizados
path = Path("/content/int")

In [25]:
# Criar um DataLoader para a etapa de ajuste fino

vocab = learn.dls.vocab
dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True, vocab=dls_train.vocab),
    get_items=get_txt, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [26]:
dls_lm.show_batch(max_n=2)

,text,text_
0,xxbos xxmaj hey there from sunny xxmaj xxunk xxmaj what 's the deal with your products xxbos xxmaj farewell xxunk of consumer satisfaction xxmaj till next time xxunk xxbos xxmaj hey there xxunk xxmaj what 's up i was just wondering how your day went xxbos xxmaj random thought – xxmaj i xxunk in need of new shoes that can keep up with my xxunk lifestyle xxmaj any recommendations xxbos xxmaj hey partner xxmaj i xxunk got a xxunk for,xxmaj hey there from sunny xxmaj xxunk xxmaj what 's the deal with your products xxbos xxmaj farewell xxunk of consumer satisfaction xxmaj till next time xxunk xxbos xxmaj hey there xxunk xxmaj what 's up i was just wondering how your day went xxbos xxmaj random thought – xxmaj i xxunk in need of new shoes that can keep up with my xxunk lifestyle xxmaj any recommendations xxbos xxmaj hey partner xxmaj i xxunk got a xxunk for some
1,xxunk an engineer with a xxunk for technology xxmaj how about you xxbos xxmaj xxunk there xxmaj i xxunk after a pair of shoes xxunk in black xxbos xxmaj xxunk fabulous souls xxmaj green shirts are on my radar and xxmaj i xxunk wondering if there 's a chance for a xxunk option xxbos xxmaj hi xxunk i picked this up earlier but i think i want to return it xxmaj is that cool xxbos xxmaj excuse me xxmaj i,an engineer with a xxunk for technology xxmaj how about you xxbos xxmaj xxunk there xxmaj i xxunk after a pair of shoes xxunk in black xxbos xxmaj xxunk fabulous souls xxmaj green shirts are on my radar and xxmaj i xxunk wondering if there 's a chance for a xxunk option xxbos xxmaj hi xxunk i picked this up earlier but i think i want to return it xxmaj is that cool xxbos xxmaj excuse me xxmaj i xxunk


In [27]:
# Criar modelo com dataset de interações comerciais
learn_fine = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3,
    metrics=[accuracy, Perplexity()]).to_fp16()

In [28]:
# Carregar os pesos do modelo de previsão de artigos da wikipedia
learn_fine = learn.load_encoder('colab_encoder_01')

In [29]:
# Ajuste fino
learn_fine.fit_one_cycle(5, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,2.623832,3.148892,0.404847,23.310215,02:12
1,2.652626,3.163655,0.403420,23.656910,02:05
2,2.595777,3.173937,0.402769,23.901402,02:21
3,2.555416,3.178530,0.402386,24.011419,02:05
4,2.536613,3.179537,0.402311,24.035624,02:08


In [30]:
# Salvar o modelo ajustado
learn_fine.save_encoder('colab_encoder_mrkt_01')

### Treinando o classificador

In [31]:
# Criar um dataloader para o treinamento do classificador
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['BUY', 'REFUND', 'LIST', 'GREETING', 'GOODBYE']),
    splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [32]:
dls_clas.show_batch(max_n=3)

,text,category
0,xxbos xxmaj hey music xxunk xxmaj i xxunk on a sonic crusade in pursuit of xxunk that 'll xxunk my auditory senses in a cocoon of auditory xxunk i xxunk for melodies that 'll xxunk me like a symphony beats that 'll xxunk with my heart 's rhythm and audio quality so xxunk it 'll make angels xxunk xxmaj can you be the virtuoso who guides me toward this auditory nirvana,BUY
1,xxbos xxmaj xxunk dear human xxmaj i xxunk got a grand plan for a picnic fiesta and xxmaj i xxunk xxunk on a xxunk that 'll be the stuff of legends xxmaj it needs to be a xxunk a xxunk a fruit sensation that 'll make the world rethink its stance on produce xxmaj can you aid me in this monumental quest for the ultimate xxunk,BUY
2,xxbos xxmaj xxunk kind souls xxmaj i xxunk currently embarked upon a xxunk adventure of epic proportions one that centers around the pursuit of the quintessential xxunk xxmaj an endeavor of this magnitude requires the guidance of xxunk experts such as xxunk xxmaj pray share your wisdom and aid me in selecting the fruit that shall reign as the crown jewel of my culinary xxunk,BUY


In [33]:
# Criar modelo para o treinamento do classificador
learnClas = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5,
                                metrics=accuracy).to_fp16()

In [35]:
# Carregar os pesos do modelo de previsão de interações comerciais
learnClas= learnClas.load_encoder('colab_encoder_mrkt_01')

A partir deste ponto acontece algo interessante. O ajuste fino será agora realizado congelando algumas das camadas do modelo em diferentes épocas do treinamento.

In [36]:
learnClas.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.322704,1.096562,0.656716,00:01


In [37]:
learnClas.freeze_to(-2)
learnClas.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,1.088184,0.704936,0.917910,00:03


In [38]:
learnClas.freeze_to(-3)
learnClas.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.955224,0.560451,0.850746,00:02


In [39]:
learnClas.unfreeze()
learnClas.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.822754,0.446960,0.925373,00:01
1,0.785573,0.379953,0.947761,00:01


Aqui temos enfim nosso modelo de processamento de linguagem natural capaz de classificar interações comerciais do ponto de vista do cliente.

In [46]:
# Teste de previsão
predict = learnClas.predict("I only have $5, can i buy a banana?")
predict

('BUY', tensor(0), tensor([0.8441, 0.0642, 0.0183, 0.0240, 0.0494]))

In [43]:
# Exportar classificador
learnClas.export('colab_complete_model_vs1.0')